In [25]:
from datetime import datetime
import requests  # type: ignore

def convert_temperature(temp_kelvin, unit):
    """
    Convert temperature from Kelvin to Celsius or Fahrenheit.
    """
    if unit.upper() == "C":
        return f"{temp_kelvin - 273.15:.2f}°C"
    elif unit.upper() == "F":
        return f"{(temp_kelvin - 273.15) * 9/5 + 32:.2f}°F"
    return f"{temp_kelvin:.2f}K"


def wind_degree_to_direction(degree):
    """
    Convert wind degree to cardinal direction.
    """
    directions = [
        "North", "North-Northeast", "Northeast", "East-Northeast", "East",
        "East-Southeast", "Southeast", "South-Southeast", "South", "South-Southwest",
        "Southwest", "West-Southwest", "West", "West-Northwest", "Northwest", "North-Northwest"
    ]
    idx = int((degree + 11.25) / 22.5)
    return directions[idx % 16]


def unix_timestamp_to_localtime(timestamp, timezone):
    """
    Convert Unix timestamp to local time.
    """
    local_time = datetime.fromtimestamp(timestamp + timezone)
    return local_time.strftime("%I:%M:%S %p")


def fetch_weather(api_key, location):
    """
    Fetch weather data from OpenWeatherMap API.
    """
    try:
        complete_api_link = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}"
        api_link = requests.get(complete_api_link)
        return api_link.json()
    except requests.exceptions.RequestException as e:
        print("Error fetching weather data:", e)
        return None


def write_to_file(weather_data, temperature_unit):
    """
    Write weather information to a text file.
    """
    try:
        with open("weatherinfo.txt", "w+") as f:
            date_time = datetime.now().strftime("%d %b %Y | %I:%M:%S %p")
            f.write("-------------------------------------------------------------\n")
            f.write(f"Weather Stats for - {weather_data['name']} | {weather_data['sys']['country']} | {date_time}\n")
            f.write("-------------------------------------------------------------\n")

            f.write(f"\tCurrent temperature is : {convert_temperature(weather_data['main']['temp'], temperature_unit)}\n")
            f.write(f"\tCurrent weather desc   : {weather_data['weather'][0]['description']}\n")
            f.write(f"\tCurrent Humidity       : {weather_data['main']['humidity']} %\n")
            f.write(f"\tCurrent wind speed     : {weather_data['wind']['speed']} km/h \n")
            f.write(f"\tCurrent wind direction : {wind_degree_to_direction(weather_data['wind']['deg'])} \n")
            f.write(f"\tToday's sunrise time   : {unix_timestamp_to_localtime(weather_data['sys']['sunrise'], weather_data['timezone'])} \n")
            f.write(f"\tToday's sunset time    : {unix_timestamp_to_localtime(weather_data['sys']['sunset'], weather_data['timezone'])} \n")

        print("Weather information written to weatherinfo.txt")
    except IOError as e:
        print("Error writing to file:", e)


def main():
    """
    Main function to run the weather information app.
    """
    print("Welcome to the Weather Information App!")
    print("You need an API key to access weather data from OpenWeatherMap.")
    print("You can obtain your API key by signing up at https://home.openweathermap.org/users/sign_up")

    api_key = input("Please enter your OpenWeatherMap API key: ")
    location = input("Enter the city name: ")
    temperature_unit = input("Enter the temperature unit. 'C' for Celsius and 'F' for Fahrenheit: ")

    if temperature_unit.upper() not in ["C", "F"]:
        print("Temperature unit must either be 'C' or 'F'.")
        return

    weather_data = fetch_weather(api_key, location)

    if weather_data:
        if weather_data["cod"] == "401":
            print("Invalid API key.")
            return
        elif weather_data["cod"] == "404":
            print("City not found.")
            return

        write_to_file(weather_data, temperature_unit)

        print(f"Current City          : {weather_data['name']}, {weather_data['sys']['country']}")
        print(f"Current temperature is: {convert_temperature(weather_data['main']['temp'], temperature_unit)}")
        print